In [ ]:
import geopandas as gpd

import sys
from pathlib import Path
PROJECT_ROOT = Path.cwd().parent# 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(PROJECT_ROOT)
sys.path.append(str(PROJECT_ROOT))

from matplotlib.font_manager import fontManager
import matplotlib as mlp
FONT_PATH = PROJECT_ROOT / "ChineseFont.ttf"
fontManager.addfont(str(FONT_PATH))
mlp.rcParams["font.family"] = "ChineseFont"

import warnings
warnings.filterwarnings('ignore')

In [ ]:

from utils.helper_grid_s1 import build_projected_grid_generator, plot_local_grid_by_targets

In [ ]:
# 使用產生器逐縣市建立方格
shp_path = r"C:\labs\geo-grid\SHP\TOWN-TW97-LL\TOWN_MOI_1140318.shp"
output_dir = Path(r"C:\labs\geo-grid\data\output_grids\grids_300m")
output_dir.mkdir(parents=True, exist_ok=True)

# 1. 呼叫函式
# 注意：這一行執行速度極快，因為它還沒開始算，只是準備好 "產生器"
gdf_tm2, grid_gen = build_projected_grid_generator(
    shp_path,
    cell_size=300,
    clip_mode="filter",
    id_scheme="xy"
)

gdf_out_path = output_dir / f"gdf_tm2.parquet"
gdf_tm2.to_parquet(gdf_out_path)
print(f"行政區資料準備完成，共 {len(gdf_tm2)} 筆。開始網格生成迴圈...")

# 2. 開始 "消費" 產生器
# 只有跑這個迴圈時，才會依序計算台北、新北、桃園...
for county_name, local_grid in grid_gen:
    
    # 這裡你可以拿到「當下那個縣市」的網格
    print(f"-> 收到: {county_name}, 格數: {len(local_grid):,}")
    
    # 動作：存成 Parquet
    out_path = output_dir / f"Grid_{county_name}_300m.parquet"
    local_grid.to_parquet(out_path)
    
    # 動作：或者做一些簡單分析
    # density = len(local_grid) / some_area
    
    # 迴圈跑完這一次，local_grid 就會被銷毀，記憶體釋放

print("全部作業完成！")

In [ ]:
save_path_gdf = r"C:\labs\geo-grid\data\output_grids\grids_100m\gdf_tm2.parquet"
gdf_tm2_loaded = gpd.read_parquet(save_path_gdf)

In [ ]:
save_path_grid = r"C:\labs\geo-grid\data\output_grids\grids_100m\Grid_臺中市_100m.parquet"
grid_north_loaded = gpd.read_parquet(save_path_grid)

In [ ]:
gdf_tm2_loaded

In [ ]:
grid_north_loaded

In [ ]:
# 檢視特定區域的方格
local_grid = plot_local_grid_by_targets(
    figsize=(14, 16),
    town_gdf=gdf_tm2_loaded,
    grid_gdf=grid_north_loaded,
    targets=[
        ("臺中市", '北區'),
        ("臺中市", '西屯區'),
        ("臺中市", '南屯區')

    ],
    show_labels=True,
    town_linewidth=0.03,
    grid_linewidth=0.1,
    rasterized=False
)

In [ ]:
# import pandas as pd
# import geopandas as gpd
# from pathlib import Path
# import warnings

# # 忽略一些 appending 的警告
# warnings.filterwarnings('ignore')

# source_dir = Path(r"C:\labs\geo-grid\OUTPUT_GRIDS")
# output_file = r"C:\labs\geo-grid\All_Taiwan_Grid.gpkg"  # 存成 GeoPackage

# # 取得所有 parquet 檔
# files = list(source_dir.glob("*.parquet"))

# print(f"開始合併 {len(files)} 個檔案到 {output_file}...")

# for i, file in enumerate(files):
#     # 1. 讀取一個縣市
#     gdf = gpd.read_parquet(file)
    
#     # 2. 寫入 (如果是第一個檔就 'w' 寫入，之後都用 'a' 附加)
#     mode = 'w' if i == 0 else 'a'
    
#     # 注意：to_file 支援 append 模式，但速度會隨檔案變大而變慢
#     gdf.to_file(output_file, driver="GPKG", mode=mode)
    
#     print(f"已合併: {file.name} ({i+1}/{len(files)})")

# print("合併完成！")